# 16) Data Manipulation with Pandas

Related references:

- [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/index.html)
- [Python for Data Analysis, 2nd Edition](https://www.safaribooksonline.com/library/view/python-for-data/9781491957653/) 

### First, let's check that we have Pandas installed

Previous lectures had instructions to install Pandas. Let's check if we are good to go!

In [ ]:
# Just as NumPy is commonly abbreviated as 'np', Pandas is abbreviated as 'pd'
import pandas as pd

pd.__version__

## Pandas extends features of NumPy

Pandas is a new library built on NumPy. Ever wish you could have column and row labels on a NumPy `ndarray`? The Pandas `DataFrame` answers that wish, and can also handle heterogeneous types and/or missing data! 

The three main data types in Pandas are built on the Numpy `ndarray`:

- `Series`
- `DataFrame`
- `Index`

We'll also be using NumPy later, so we'll import that now.

In [ ]:
import numpy as np

### The Pandas `Series` Object

A `Series` is a one-dimensional array-like object containing a sequence of values (of similar types to NumPy types) and an associated array of data labels, called its *index*. The simplest Series is formed from only an array of data:

In [ ]:
obj = pd.Series([4, 7, -5, 3])
obj

The string representation of a Series displayed interactively shows the index on the left and the values on the right. Since we did not specify an index for the data, a default one consisting of the integers 0 through N - 1 (where N is the length of the data) is created. You can get the array representation and index object of the Series via its values and index attributes, respectively:

In [ ]:
obj.values

In [ ]:
obj.index  # like range(4)

Often it will be desirable to create a Series with an index identifying each data point with a label:

In [ ]:
obj2 = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])
obj2

In [ ]:
obj2.index

Like with a NumPy array, data can be accessed by the associated index via the familiar Python square-bracket notation:

In [ ]:
obj2[1:3]

Compared with NumPy arrays, you can use labels in the index when selecting single values or a set of values:

In [ ]:
obj2['a']

In [ ]:
obj2['d'] = 6
obj2[['c', 'a', 'd']]

Here ['c', 'a', 'd'] is interpreted as a list of indices, even though it contains strings instead of integers.

Using NumPy functions or NumPy-like operations, such as filtering with a boolean array, scalar multiplication, or applying math functions, will preserve the index-value link:

In [ ]:
obj2[obj2 > 0]

In [ ]:
obj2 * 2

In [ ]:
np.exp(obj2)

Another way to think about a Series is as a fixed-length, ordered dict, as it is a mapping of index values to data values. It can be used in many contexts where you might use a dict:

In [ ]:
'b' in obj2

In [ ]:
'e' in obj2

Should you have data contained in a Python dict, you can create a Series from it by passing the dict:

In [ ]:
sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
obj3 = pd.Series(sdata)
obj3

When you are only passing a dict, the index in the resulting Series will have the dict’s keys in sorted order. You can override this by passing the dict keys in the order you want them to appear in the resulting Series:

In [ ]:
states = ['Texas', 'California', 'Ohio', 'Oregon']
obj4 = pd.Series(sdata, index=states)
obj4

In [ ]:
obj4.values

Here, three values found in sdata were placed in the appropriate locations, but since no value for `California` was found, it appears as NaN (not a number), which is considered in pandas to mark missing or NA values. Since 'Utah' was not included in states, it is excluded from the resulting object.

The isnull and notnull functions in pandas should be used to detect missing data:

In [ ]:
pd.isnull(obj4)

In [ ]:
pd.notnull(obj4)

Series also has these as instance methods:

In [ ]:
obj4.isnull()

A useful Series feature for many applications is that it automatically aligns by index label in arithmetic operations:

In [ ]:
obj3

In [ ]:
obj4

In [ ]:
obj3 + obj4

Both the Series object itself and its index have a name attribute, which integrates with other key areas of pandas functionality:

In [ ]:
obj4.name = 'population'
obj4.index.name = 'state'
obj4

A Series’s index can be altered in-place by assignment:

In [ ]:
obj

In [ ]:
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']
obj

### The Pandas `DataFrame` Object

A DataFrame represents a rectangular table of data and contains an ordered collection of columns, each of which can be a different value type (numeric, string, boolean, etc.). The DataFrame has both a row and column index; it can be thought of as a dict of Series all sharing the same index. Under the hood, the data is stored as one or more two-dimensional blocks rather than a list, dict, or some other collection of one-dimensional arrays. 

*FYI*: While a DataFrame is physically two-dimensional, you can use it to represent higher dimensional data in a tabular format using hierarchical indexing, which we'll come back to later.

There are many ways to construct a `DataFrame`, though one of the most common is from a dict of equal-length lists or NumPy arrays:

In [ ]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],
        'year': [2000, 2001, 2002, 2001, 2002, 2003],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
frame = pd.DataFrame(data)

The resulting DataFrame will have its index assigned automatically as with Series, and the columns are placed in sorted order:

In [ ]:
frame

*FYI*: Thank Jupyter notebooks for the nice DataFrame output format.

For large DataFrames, the head method selects only the first five rows:

In [ ]:
frame.head()

If you pass a column that isn't contained in the dict, it will appear with missing values in the result:

In [ ]:
frame2 = pd.DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
                      index=['one', 'two', 'three', 'four',
                             'five', 'six'])
frame2                      

A column in a DataFrame can be retrieved as a Series either by dict-like notation or by attribute:

In [ ]:
frame2['state']

Rows can also be retrieved by position or name with the special `loc` attribute:

In [ ]:
frame2.loc['three']

Columns can be modified by assignment. For example, the empty 'debt' column could be assigned a scalar value or an array of values:

In [ ]:
frame2['debt'] = 16.5
frame2

In [ ]:
frame2['debt'] = np.arange(6.)
frame2

When you are assigning lists or arrays to a column, the value’s length must match the length of the DataFrame. If you assign a Series, its labels will be realigned exactly to the DataFrame’s index, inserting missing values in any holes:

In [ ]:
val = pd.Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five'])
frame2['debt'] = val
frame2

In [ ]:
val2 = pd.Series([-1.2, -1.5, -1.7], index=['one', 'three', 'six'])
frame2['debt'] = val
frame2

Assigning a column that doesn’t exist will create a new column. The `del` keyword will delete columns as with a dict.

As an example of `del`, let's add a new column of boolean values where the state column equals 'Ohio'

In [ ]:
frame2['eastern'] = frame2['state'] == 'Ohio'
frame2

The `del` method can then be used to remove this column:

In [ ]:
del frame2['eastern']
frame2.columns

*Caution*: The column returned from indexing a DataFrame is a view on the underlying data, not a copy. Thus, any in-place modifications to the Series will be reflected in the DataFrame. The column can be explicitly copied with the Series’s `copy` method.

Another common form of data is a nested dict of dicts:

In [ ]:
pop = {'Nevada': {2001: 2.4, 2002: 2.9},
       'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}

If the nested dict is passed to the DataFrame, pandas will interpret the outer dict keys as the columns and the inner keys as the row indices:

In [ ]:
frame3 = pd.DataFrame(pop)
frame3

#### From a two-dimensional NumPy array

Given a two-dimensional array of data, we can create a ``DataFrame`` with any specified column and index names.
If omitted, an integer index will be used for each:

In [ ]:
pd.DataFrame(np.random.rand(3, 2),
             columns=['foo', 'bar'],
             index=['a', 'b', 'c'])

You can transpose the DataFrame (swap rows and columns) with similar syntax to a NumPy array:

In [ ]:
frame3.T

The keys in the inner dicts are combined and sorted to form the index in the result. This isn’t true if an explicit index is specified:

In [ ]:
pd.DataFrame(pop, index=[2001, 2002, 2003])

Dicts of Series are treated in much the same way:

In [ ]:
pdata = {'Ohio': frame3['Ohio'][:-1],
         'Nevada': frame3['Nevada'][:2]}
pd.DataFrame(pdata)

If a DataFrame’s index and columns have their name attributes set, these will also be displayed:

In [ ]:
frame3.index.name = 'year'
frame3.columns.name = 'state'
frame3

As with Series, the values attribute returns the data contained in the DataFrame as a two-dimensional ndarray:

In [ ]:
frame3.values

In [ ]:
type(frame3.values)

If the DataFrame’s columns are different dtypes, the dtype of the values array will be chosen to accommodate all of the columns:

In [ ]:
frame2.values

In [ ]:
type(frame2.values)

### The Pandas `Index` Object

Index objects are responsible for holding the axis labels and other metadata (like the axis name or names). Any array or other sequence of labels you use when constructing a Series or DataFrame is internally converted to an Index:

In [ ]:
obj = pd.Series(range(3), index=['a', 'b', 'c'])
index = obj.index
index

In [ ]:
index[1:]

Index objects are immutable and thus can’t be modified by the user:

In [ ]:
index[1] = 'd'  # TypeError

Immutability makes it safer to share Index objects among data structures:

In [ ]:
labels = pd.Index(np.arange(3))
labels

In [ ]:
obj2 = pd.Series([1.5, -2.5, 0], index=labels)
obj2

In addition to being array-like, an Index also behaves like a fixed-size set:

In [ ]:
frame3

In [ ]:
frame3.columns

In [ ]:
'Ohio' in frame3.columns

In [ ]:
2003 in frame3.index

*Caution:* Unlike Python sets, a pandas Index can contain duplicate labels:

In [ ]:
dup_labels = pd.Index(['foo', 'foo', 'bar', 'bar'])
dup_labels

Selections with duplicate labels will select all occurrences of that label.

### Some `Index` methods and properties


|Method | Description |
|-------------|-----------------------------|
|append       | Concatenate with additional Index objects, producing a new Index |
|difference   | Compute set difference as an Index|
|intersection | Compute set intersection    |
|union        | Compute set union           |
|isin         | Compute boolean array indicating whether each value is contained in the passed collection |
|delete       | Compute new Index with element at index i deleted |
|drop         | Compute new Index by deleting passed values |
|insert       | Compute new Index by inserting element at index i |
|is_monotonic | Returns True if each element is greater than or equal to the previous element |
|is_unique    | Returns True if the Index has no duplicate values |
|unique       | Compute the array of unique values in the Index |

For example:

In [ ]:
indA = pd.Index([1, 3, 5, 7, 9])
indB = pd.Index([2, 3, 5, 7, 11])
indA.is_monotonic

In [ ]:
indA.intersection(indB)

## Essential Functionality

As throughout this class, we’ll focus on the most important features, leaving the less common (i.e., more esoteric) things for you to explore on your own.

### Reindexing

An important method on pandas objects is `reindex`, which means to create a new object with the data *conformed* to a new index. Consider an example:

In [ ]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

Calling `reindex` on this Series rearranges the data according to the new index, introducing missing values if any index values were not already present:

In [ ]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

For ordered data like time series, it may be desirable to do some interpolation or filling of values when reindexing. The method option allows us to do this, using a method such as `ffill`, which forward-fills the values:

In [ ]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3

In [ ]:
obj3.reindex(range(6), method='ffill')

In [ ]:
# We can also backfill
obj3.reindex(range(6), method='bfill')

In [ ]:
# or fill with a specified value
obj3.reindex(range(6), fill_value='white')

With DataFrame, reindex can alter either the (row) index, columns, or both. When passed only a sequence, it reindexes the rows in the result:

In [ ]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=['a', 'c', 'd'],
                     columns=['Ohio', 'Texas', 'California'])
frame

In [ ]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

The columns can be reindexed with the columns keyword:

In [ ]:
states = ['Texas', 'Utah', 'California']
frame.reindex(columns=states)

### Dropping Entries from an Axis

Dropping one or more entries from an axis is easy if you already have an index array or list without those entries. As that can require a bit of munging and set logic, the drop method will return a new object with the indicated value or values deleted from an axis:

In [ ]:
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
obj

In [ ]:
new_obj = obj.drop('c')
new_obj

In [ ]:
obj.drop(['d', 'c'])

In [ ]:
obj

With DataFrame, index values can be deleted from either axis. To illustrate this, we first create an example DataFrame:

In [ ]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

Calling `drop` with a sequence of labels will drop values from the row labels (axis 0):

In [ ]:
data.drop(['Colorado', 'Ohio'])

You can drop values from the columns by passing axis=1 or axis='columns':

In [ ]:
data.drop('two', axis=1)

In [ ]:
data.drop(['two', 'four'], axis='columns')

Many functions, like drop, which modify the size or shape of a Series or DataFrame, can manipulate an object *in-place* without returning a new object:

In [ ]:
obj.drop('c', inplace=True)
obj

Be careful with the inplace, as it destroys any data that is dropped.

## Indexing, Selection, and Filtering

Series indexing (obj[...]) works analogously to NumPy array indexing, except you can use the Series’s index values instead of only integers. Here are some examples of this:

In [ ]:
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj

In [ ]:
obj['b']

In [ ]:
obj[1]

In [ ]:
obj[2:4]

In [ ]:
obj[['b', 'a', 'd']]

In [ ]:
obj[[1, 3]]

In [ ]:
obj[obj < 2]

*Caution*: Slicing with labels behaves differently than normal Python slicing in that the endpoint is inclusive:

In [ ]:
obj['b':'c']

*Setting* using these methods modifies the corresponding section of the Series:

In [ ]:
obj['b':'c'] = 5
obj

Indexing into a DataFrame is for retrieving one or more columns either with a single value or sequence:

In [ ]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

In [ ]:
data['two']

In [ ]:
data[['three', 'one']]

### Selection with `loc` and `iloc`

These slicing and indexing conventions can be a source of confusion.
For example, if your ``Series`` has an explicit integer index, an indexing operation such as ``data[1]`` will use the explicit indices, while a slicing operation like ``data[1:3]`` will use the implicit Python-style index.

In [ ]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data

In [ ]:
# explicit index when indexing
data[1]

In [ ]:
# implicit index when slicing
data[1:3]

Because of this potential confusion in the case of integer indexes, Pandas provides some special *indexer* attributes that explicitly expose certain indexing schemes.
These are not functional methods, but attributes that expose a particular slicing interface to the data in the ``Series``.

First, the ``loc`` attribute allows indexing and slicing that always references the explicit index:

In [ ]:
data.loc[1]

In [ ]:
data.loc[1:3]

The ``iloc`` attribute allows indexing and slicing that always references the implicit Python-style index:

In [ ]:
data.iloc[1]

In [ ]:
data.iloc[1:3]

One guiding principle of Python code is that "explicit is better than implicit."
The explicit nature of ``loc`` and ``iloc`` make them very useful in maintaining clean and readable code; especially in the case of integer indexes, I recommend using these both to make code easier to read and understand, and to prevent subtle bugs due to the mixed indexing/slicing convention.

**Next up**: Data selection in dataframes!